In [ ]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

In [ ]:
I = cv2.imread('lego_2.png')
I_rgb = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)

if I is None:
    print("Error: Could not load image. Check file path and format.")
    exit()
    
plt.imshow(I_rgb)
plt.show()

In [ ]:
# Grayscale + blur
I_gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
I_blur = cv2.GaussianBlur(I_gray, (5,5), 0)

# Adaptive threshold
#Converted to black and white
B = cv2.adaptiveThreshold(
    I_blur, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV,
    19, 2
)

# Morphology
kernel = np.ones((3,3), np.uint8)
#close = fill small holes
#open = removes small white noise outside objects
B_clean = cv2.morphologyEx(B, cv2.MORPH_CLOSE, kernel, iterations=2) 
B_clean = cv2.morphologyEx(B_clean, cv2.MORPH_OPEN, kernel, iterations=1)

# Find contours
contours, hierarchy = cv2.findContours(B_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours by area to get only LEGO blocks
min_area = 800
lego_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

print(f"Total number of LEGO blocks detected: {len(lego_contours)}")

# Create filled mask visualization
I_filled = np.ones_like(I_rgb) * 255  # white background
cv2.drawContours(I_filled, lego_contours, -1, (0, 0, 0), thickness=cv2.FILLED)

plt.figure(figsize=(12,6))
plt.subplot(1,3,1)
plt.imshow(I_rgb)
plt.title('Original Image')

plt.subplot(1,3,2)
plt.imshow(I_gray, cmap='gray')
plt.title("Gray Scale")

plt.subplot(1,3,3)
plt.imshow(I_filled)
plt.title(f"LEGO Blocks Filled Mask ({len(lego_contours)} blocks)")
plt.show()

In [ ]:
# Calculate grid size for displaying all blocks
num_blocks = len(lego_contours)
cols = 4  # Display 4 blocks per row
rows = (num_blocks + cols - 1) // cols  # Calculate needed rows
plt.figure(figsize=(15, 3*rows))

#Looping through each LEGO block
for idx, contour in enumerate(lego_contours, 1):
    # Get bounding box
    x, y, w, h = cv2.boundingRect(contour)
    
    # Add padding
    padding = 5
    y_min = max(0, y - padding) #Don't go below 0
    y_max = min(I_rgb.shape[0], y + h + padding) #Image height cannot be exceeded
    x_min = max(0, x - padding) #Don't go left of 0
    x_max = min(I_rgb.shape[1], x + w + padding) #Image width cannot be exceeded
    
    # Create mask directly in the cropped region size
    mask_region = np.zeros((y_max - y_min, x_max - x_min), dtype=np.uint8)
    
    # Adjust contour coordinates to the cropped region
    contour_shifted = contour - [x_min, y_min]
    cv2.drawContours(mask_region, [contour_shifted], -1, 255, thickness=cv2.FILLED)
    
    # Extract the block region from original image
    block_original = I_rgb[y_min:y_max, x_min:x_max].copy()
    
    # Create a white background version
    block_display = np.ones_like(block_original) * 255
    block_display[mask_region > 0] = block_original[mask_region > 0]
    
    # Display
    plt.subplot(rows, cols, idx)
    plt.imshow(block_display)
    plt.title(f"Block {idx}")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
#test commit :)

In [ ]:
print('Test')